Ref: https://htmlpreview.github.io/?https://github.com/CogStack/MedCATtutorials/blob/main/notebooks/introductory/Part_4_3_Annotating_documents_with_the_full_MedCAT_pipeline_with_MetaAnnotations.html

File previously called - LatestFeb15-MedCAT_Tutorial___Part_4_3_Annotating_documents_with_the_full_MedCAT_pipeline_with_MetaAnnotations.ipynb

In [ ]:
####################
########SKIP########
####################

# Install medcat - ignore this and pip install in next cell
! pip install medcat==1.5.0
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()

In [ ]:
####################
########SKIP########
####################

!pip install medcat #no need to run every time

In [ ]:
import medcat

In [ ]:
medcat.__version__ #will be 1.5.0

In [ ]:
####################
########SKIP########
####################

#skip this
!pip uninstall -y spacy
!pip install spacy==3.1.0

In [ ]:
####################
########SKIP########
####################

#skip this
!pip uninstall -y blis
!pip install blis==0.7.5

In [ ]:
!pip install -U numpy

In [ ]:
import spacy
spacy.__version__ #will be 3.1.0

**Restart the runtime if on colab, sometimes necessary after installing models**

In [ ]:
from platform import python_version

print(python_version()) #will be 3.10.5

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import json

from matplotlib import pyplot as plt
from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.config import Config
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT
from medcat.preprocessing.tokenizers import TokenizerWrapperBPE
from tokenizers import ByteLevelBPETokenizer

from tqdm.autonotebook import tqdm, trange

In [ ]:
#jc - i want another model - snomed is snomed international doesnot have type id so try umls which includes snomed us and type id

model_pack_path = "models/mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5.zip"
#model_pack_path = "models/umls_self_train_model_e6a3660d3ba19f52.zip"

In [ ]:
#skip this

DATA_DIR = "./data_p4.3/"
MODEL_DIR = "./models/"

#model_pack_path = MODEL_DIR + "medmen_wstatus_2021_oct.zip"

#jc - i want another model - snomed

model_pack_path = "models/mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5.zip"
#model_pack_path = "models/umls_self_train_model_e6a3660d3ba19f52.zip"

In [ ]:
MODEL_DIR = 'models/'

In [ ]:
DATA_DIR = "data_p4.3/"

In [ ]:
####################
########SKIP########
####################

# Download the models and required data - this gives an error but still downloads the json file. Don't need data i.e. pt_notes.csv - download manually and save in folders

# Download the models and required data
#!wget https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/pt_notes.csv -P ./data/
!wget https://raw.githubusercontent.com/CogStack/MedCATtutorials/main/notebooks/introductory/data/MedCAT_Export.json -P ./data/

In [ ]:
####################
########SKIP########
####################

# Download the medcat modelpack - giving an error for snomed pack
#!wget https://medcat.rosalind.kcl.ac.uk/media/medmen_wstatus_2021_oct.zip -P ./models/

#why is this not working? do i need it? I have the files in the folder ie meta_status, cdb etc.
#!wget https://medcat.rosalind.kcl.ac.uk/media/mc_modelpack_snomed_int_16_mar_2022_25be3857ba34bdd5.zip -P ./models/

!wget https://medcat.rosalind.kcl.ac.uk/media/umls_self_train_model_e6a3660d3ba19f52.zip -P ./models/

In [ ]:
#this works even though the previous two cells gave errors

# Load model pack and Create CAT - the main class from medcat used for concept annotation
# Create the full pipeline with models for meta-annotations
cat = CAT.load_model_pack(model_pack_path)

In [ ]:
#another useful ref - https://github.com/tomolopolis/SNOMED-CT_Analysis/blob/master/Exploring%20a%20SNOMED-CT%20Release.ipynb

In [ ]:
####################
########SNOMED MODEL########
####################

# Detect only Disease and Mental Disorders
# |T047|Disease or Syndrome
# |T048|Mental or Behavioral Dysfunction

#JC - try getting only body part structure type ID - SCTID: 52530000, TUI T-01000 Body Structure. T-02000	Clinical finding (finding).
#TRY: Unique Identifier: T029 on umls model for anatomy - this is working with umls model
#https://uts.nlm.nih.gov/uts/umls/semantic-network/T029
#This is not working - doesn't recognise the type ID even when I use the medmen one or snomed ones
#type_ids_filter = ['T047', 'T048']
#type_ids_filter = ['T029'] #T029 for anatomy - works with umls model
#type_ids_filter = ['67667581', '9090192'] #Try 67667581 for finding and 9090192 for disorder - works with snomed model
type_ids_filter = ['37552161'] #body structure snomed
cui_filters = set()
for type_ids in type_ids_filter:
    cui_filters.update(cat.cdb.addl_info['type_id2cuis'][type_ids])
cat.cdb.config.linking['filters']['cuis'] = cui_filters

A subset of type_ids for the MedCAT 1.2 model based on SNOMED-CT

81102976  : organism  
91187746  : substance
28321150  : procedure
37552161  : body structure
9090192   : disorder  
67667581  : finding   
7882689   : qualifier value
33782986  : morphologic abnormality
32816260  : physical object
91776366  : product   
31601201  : person    
75168589  : environment
2680757   : observable entity
43039974  : attribute
82417248  : navigational concept
13371933  : social concept
337250    : specimen  
90170645  : record artifact
40357424  : foundation metadata concept
29422548  : core metadata concept
87776218  : role

https://discourse.cogstack.org/t/using-type-ids-with-the-snomedct-model/165/2


In [ ]:
####################
########UMLS MODEL########
####################

# JC - Detect only Pain concepts - using Type IDs from gold standard - will have to manually look through and remove any unwanted ones- works umls model
#if using snomed- try T-02000	Clinical finding (finding)

type_ids_filter = ['T011',
'T014',
'T018',
'T020',
'T029',
'T030',
'T033',
'T039',
'T040',
'T042',
'T049']
#t033 giving lot of vague terms like offering, engagement. It is semantic type Finding
cui_filters = set()
for type_ids in type_ids_filter:
    cui_filters.update(cat.cdb.addl_info['type_id2cuis'][type_ids])
cat.cdb.config.linking['filters']['cuis'] = cui_filters

In [ ]:
#!wget -N https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/pt_notes.csv -P $DATA_DIR

## Document annotation

The following is a replica of the document annotation code done in [Part 3.2.](https://colab.research.google.com/drive/1q29RbHlZoFK7TcvMKITi3ABbE-E_fw30), with the only change that we have meta-annotations in the pipeline now.

In [ ]:
####################
########SKIP########
####################

# Load the data
#data = pd.read_csv(DATA_DIR + "pt_notes.csv")
#data.head()

In [ ]:
#jc pain notes

#data = pd.read_csv('classified_as_pain_sapbert_CE.csv')
data = pd.read_csv('classified_as_anatomy_sapbert_CE.csv')

In [ ]:
#jc
data.head()

In [ ]:
data = data.drop(columns=['Unnamed: 0'])
data.head()

In [ ]:
#jc
data.shape

In [ ]:
#split dataframe into batches of smaller dfs because it is taking too long to process the entire df

#df2 = data[:2000]
#df2 = data[2001:5000]
#df2 = data[5001:8000]
#df2 = data[8001:10000]
#df2 = data[10001:20000]
#df2 = data[20001:30000]
##df2 = data[20001:50000]
#df2 = data[30001:50000]
#df2 = data[50001:80000]
#df2 = data[80001:98940]
#df2 = data[60001:70000]
df2 = data[50001:65008]

In [ ]:
df2.describe()

In [ ]:
#another resource - https://colab.research.google.com/drive/1LL0EP_qN78ljs1PozaoiUDMaBo6zb4Gm
#might help incorporate docid into the annotation - still not doing it?

for umls model:

In [ ]:
#testing to see if this includes cn_doc_id in results - this works with umls model
in_data = []
for CN_Doc_ID, row in df2[['CN_Doc_ID','clean_text']].iterrows():
    text = row['clean_text']
    CN_Doc_ID = row['CN_Doc_ID']

    in_data.append((CN_Doc_ID, text))

In [ ]:
in_data #now it has doc id and text

In [ ]:
#testing to see if this includes cn_doc_id in results - now it should work - this works with umls model, not snomed

# Set the batch size to the number of documents
#batch_size = 100 # Batch size (BS) in number of documents
batch_size_chars = 500000 # Batch size (BS) in number of characters

# Run model - testing on few data first - remove [:x] for it to run on all of it
results = cat.multiprocessing_pipe(in_data[:500], # Formatted data - use in_data[:200] or less to test on small sample- whole 65k is taking too long so try in batches of 1000 as shown below
                                   batch_size = batch_size_chars, # change to batch_size = batch_size_chars if using that instead or keep batch_size=batch_size
                                   nproc=8) # Increase it when having more cores available


#in_data[:1000]

for snomed model:

In [ ]:
#jc - skip - use for snomed model - with docid
# format the df to match: required input data for multiprocessing = [(doc_id, doc_text), (doc_id, doc_text), ...]
def data_iterator(df2):
    for CN_Doc_ID, row in df2[['CN_Doc_ID', 'clean_text']].iterrows():
        text = row['clean_text']
        CN_Doc_ID = row['CN_Doc_ID']
        yield (CN_Doc_ID, str(row['clean_text']))

In [ ]:
#jc - skip - use for snomed model - without docid
# format the df to match: required input data for multiprocessing = [(doc_id, doc_text), (doc_id, doc_text), ...]
def data_iterator(df2):
    for CN_Doc_ID, row in df2[['CN_Doc_ID', 'clean_text']].iterrows():
        yield (CN_Doc_ID, str(row['clean_text']))

In [ ]:
#skip - use this for snomed model
# Set a batch size to control for the variablity between document sizes
batch_size_chars = 500000 # Batch size (BS) in number of characters

# Run model
results = cat.multiprocessing(data_iterator(df2),  # Formatted data
                              batch_size_chars = batch_size_chars,
                              nproc=8) # Number of processors

The rest is common for any model:

In [ ]:
# This will be a map from CUI to a list of documents where it appears: {"cui": [<doc_id>, <doc_id>, ...], ..}
cui_location = {}
# Let's also save the TUI location (semantic type)
type_ids_location = {}

In [ ]:
# Select only cui's with the Status Meta-annotation "Affirmed" and thier corresponding documents
for doc in list(results.keys()):
    for annotation in list(results[doc]['entities'].values()):
        if annotation['cui'] in cui_location:
            #if annotation['meta_anns']['Status']['value'] == 'Affirmed':
            cui_location[annotation['cui']].append(doc)
        else:
            #if annotation['meta_anns']['Status']['value'] == 'Affirmed':
            cui_location[annotation['cui']] = [doc]

In [ ]:
# For the type_ids and their corresponding documents
# Remember that a cui may map to more than one type_ids (one to many mapping)
for cui in cui_location.keys():
    type_ids_location[list(cat.cdb.cui2type_ids[cui])[0]] = cui_location[cui]

## Done

We have now annotated all documents in our dataset and for each CUI (Concept identifier) we know in which document it appers. We also know that all the detected entities have the status "Affirmed".

---

Please note that the number of examples I have provided is not enough to fully train the "Status" meta-annotation and one would need to provide more.

In [ ]:
####################
########SKIP########
####################

# For example, the concept with CUI: C0020538 (hypertension) appears in
set(cui_location['C1444648']) #C1444648 is code for 'offered'

In [ ]:
# Save concept location in corpus
import json
json.dump(cui_location, open("./cui_location_anatomy_25k_to_65k.json", 'w'))

In [ ]:
####################
########SKIP########
####################

results #to be formatted like when downloading annotations maybe so it is readable in a csv - JC

example output format:

{200: {'entities': {15: {'pretty_name': 'Right lower quadrant pain',
    'cui': '301754002',
    'type_ids': ['67667581'],
    'types': ['finding'],
    'source_value': 'abdominal pain (right lower quadrant',
    'detected_name': 'abdominal~pain~right~lower~quadrant',
    'acc': 1.0,
    'context_similarity': 1.0,
    'start': 185,
    'end': 221,
    'id': 15,
    'meta_anns': {'Status': {'value': 'Affirmed',
      'confidence': 0.9999881386756897,
      'name': 'Status'}}},

End of tutorial

In [ ]:
####################
########SKIP########
####################

results.keys()

In [ ]:
####################
########SKIP########
####################

import pprint

pprint.pprint(results, depth=3)

In [ ]:
#make csv of json

In [ ]:
####################
########SKIP########
####################

json.dump(results, open("./sctid_location_anatomy_with_details_50k_to_65k.json", 'w'))

In [ ]:
annotation

In [ ]:
doc

In [ ]:
annos =[]

for doc in list(results.keys()):
    docid = doc
    for annotation in list(results[doc]['entities'].values()):
        if annotation['cui'] in cui_location:
            #if annotation['meta_anns']['Status']['value'] == 'Affirmed':
            cui = annotation['cui']
            name = annotation['detected_name']
            pretty_name = annotation['pretty_name']
            type_id = annotation['type_ids']
            id2 = annotation['id']
            annos.append([docid, cui, name, pretty_name, type_id, id2])




In [ ]:
df_anno = pd.DataFrame(annos, columns=['docid','sctid','name', 'pretty_name', 'type_id', 'id2'])

In [ ]:
df_anno.head()

In [ ]:
df_anno.to_csv('sctid_with_details_for_classified_anatomy_sent_medcat_50k_to_65k.csv')